In [1]:
import os
import cohere

from langchain_pinecone import PineconeVectorStore

from langchain_cohere import CohereEmbeddings

from pinecone import Pinecone, ServerlessSpec
from llama_index.llms.cohere import Cohere

In [2]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")
LANGCHAIN_TRACING_V2 = bool(os.getenv("LANGCHAIN_TRACING_V2"))
LANGCHAIN_API_KEY = os.getenv("LANG_CHAIN_API_KEY")
pinecone_obj = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
cohere_obj = Cohere(api_key=os.getenv("COHERE_API_KEY"))
co = cohere.Client(api_key=os.getenv("COHERE_API_KEY"))

In [3]:
index_name = "bhagvat-geeta-quickstart-index"
if index_name not in pinecone_obj.list_indexes().names():
    pinecone_obj.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


embeddings = CohereEmbeddings()

In [4]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# path to an example pdf file


loader = PyPDFLoader("data/Bhagvat_Geeta.pdf")

In [6]:
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)

In [7]:
from langchain_cohere import ChatCohere
from langchain.chains.question_answering import load_qa_chain

In [8]:
def get_user_input() -> str:
    input_message = input("what do you want to ask?")
    return input_message

In [9]:
def generate_text(prompt, temp=0):
    stream = co.chat_stream(
        message=prompt, model="command-r", temperature=temp, preamble=""
    )
    for event in stream:
        if event.event_type == "text-generation":
            print(event.text, end="")

In [10]:
generate_text("What features should I consider when choosing a wireless headphone")

When choosing a wireless headphone, there are several key features to consider:

1. Sound Quality: Wireless headphones should deliver good sound quality, with clear and balanced audio. Pay attention to the driver size, frequency response, and any technologies used to enhance sound.

2. Noise Cancellation: If you plan to use your headphones in noisy environments, consider models with active noise cancellation (ANC). This feature reduces external noise, allowing you to focus on your music or calls.

3. Connectivity: Ensure the headphones support the latest wireless connectivity standards, such as Bluetooth 5.0 or later. This provides a more stable connection, faster pairing, and low latency. Check the range of the wireless connection too.

4. Battery Life: Longer battery life means you can use your headphones for extended periods without worrying about recharging. Look for models with at least 15-20 hours of continuous playback on a single charge. Also, consider the charging time and any

In [11]:
pdf_search = PineconeVectorStore.from_documents(
    texts, embedding=embeddings, index_name=index_name
)
llm = ChatCohere(temperature=0, cohere_api_key=os.getenv("COHERE_API_KEY"))
chain = load_qa_chain(llm=llm, chain_type="stuff")

In [16]:
message = get_user_input()


query = pdf_search.similarity_search(message)



result = chain.run(input_documents=query, question=message)

print(result)

As of my last update in 2023, Narendra Modi is the Prime Minister of India.
